# Diabetes class 구현

In [19]:
import numpy as np
from datetime import datetime

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

In [28]:
import numpy as np
from datetime import datetime

class Diabetes:
    def __init__(self,name,xdata,tdata,i_node,h1_node,o_node,learning_rate,iteration_count):
        self.name = name
        self.xdata = xdata
        if xdata.ndim == 1:
            self.xdata = xdata.reshape(-1,1)
            self.tdata = tdata.reshape(-1,1)
        elif xdata.ndim == 2:
            self.xdata = xdata
            self.tdata = tdata.reshape(-1,1)
        self.i_node = i_node
        self.h1_node = h1_node
        self.o_node = o_node
        self.learning_rate = learning_rate
        self.iteration_count = iteration_count
        
        self.W2 = np.random.rand(self.i_node,self.h1_node)
        self.b2 = np.random.rand(self.h1_node)
        
        self.W3 = np.random.rand(self.h1_node,self.o_node)
        self.b3 = np.random.rand(self.o_node)
    
    def loss_func(self):
        delta = 1e-7
        
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2,self.W3) + self.b3
        y = a3 = sigmoid(z3) 
        
        return -np.sum(self.tdata * np.log(y + delta) + (1 - self.tdata) * np.log((1 - y) + delta))
    
    def predict(self,test_xdata):
        z2 = np.dot(test_xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2,self.W3) + self.b3
        y = a3 = sigmoid(z3)
        
        if y >= 0.5:
            result = 1
        else:
            result = 0
        
        return y, result
    
    def accuracy(self,test_xdata,test_tdata):
        matched_list = []
        not_matched_list = []
        index_label_prediction_list = []
        for index in range(len(test_xdata)):
            (real_val, logical_val) = self.predict(test_xdata[index])
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
            index_label_prediction_list.append([index,test_tdata[index][0],logical_val])

        print("accuracy => ", len(matched_list) / len(test_xdata))
        return matched_list, not_matched_list, index_label_prediction_list
    
    def train(self):
        f = lambda x : self.loss_func()
        strat_time = datetime.now()
        for step in range(self.iteration_count):
            self.W2 -= self.learning_rate * numerical_derivative(f,self.W2)
            self.b2 -= self.learning_rate * numerical_derivative(f,self.b2)
            
            self.W3 -= self.learning_rate * numerical_derivative(f,self.W3)
            self.b3 -= self.learning_rate * numerical_derivative(f,self.b3)
            
            if (step % 400) == 0:
                print("step = ", step, "error_val = ", self.loss_func())
        end_time = datetime.now()
        print("")
        print("elasped time: ", end_time - strat_time)

In [29]:
load_data = np.loadtxt('./diabetes (1).csv',delimiter=',', dtype = np.float32)
x_data = load_data[0:500,0:-1]
t_data = load_data[0:500,[-1]]

test_x_data = load_data[501:,0:-1]
test_t_data = load_data[501:,[-1]].reshape(-1,1)

#데이터 차원 및 shape 확인
print("x_data.ndim = ", x_data.ndim,", x_data.shape = ", x_data.shape)
print("t_data.ndim = ", t_data.ndim,", t_data.shape = ", t_data.shape)

x_data.ndim =  2 , x_data.shape =  (500, 8)
t_data.ndim =  2 , t_data.shape =  (500, 1)


In [30]:
i_node = x_data.shape[1]
h1_node = 2
o_node = t_data.shape[1]

lr = 1e-3
iter_count = 10001

obj = Diabetes("Diabetes",x_data,t_data,i_node,h1_node,o_node,lr,iter_count)
obj.train()




step =  0 error_val =  341.01968111273607
step =  400 error_val =  251.27160499357288
step =  800 error_val =  245.64925105179938
step =  1200 error_val =  244.43623719395242
step =  1600 error_val =  243.73477151324545
step =  2000 error_val =  243.17441109237794
step =  2400 error_val =  242.67169377638385
step =  2800 error_val =  242.19142379831072
step =  3200 error_val =  241.69005994980267
step =  3600 error_val =  241.10705177665017
step =  4000 error_val =  240.39944434293272
step =  4400 error_val =  239.60660072459217
step =  4800 error_val =  238.82828454112934
step =  5200 error_val =  238.13430731151107
step =  5600 error_val =  237.52864926579997
step =  6000 error_val =  236.97319358217442
step =  6400 error_val =  236.4209562512928
step =  6800 error_val =  235.83434381950735
step =  7200 error_val =  235.19262304141495
step =  7600 error_val =  234.49595806719498
step =  8000 error_val =  233.76831009444282
step =  8400 error_val =  233.05803687739905
step =  8800 err

In [31]:
(matched_list,not_not_matched_list, index_label_prediction_list) = obj.accuracy(test_x_data,test_t_data)
print(index_label_prediction_list)

accuracy =>  0.8255813953488372
[[0, 1.0, 1], [1, 1.0, 1], [2, 0.0, 1], [3, 1.0, 1], [4, 1.0, 1], [5, 1.0, 1], [6, 1.0, 1], [7, 0.0, 0], [8, 0.0, 0], [9, 1.0, 1], [10, 1.0, 1], [11, 1.0, 1], [12, 1.0, 1], [13, 1.0, 1], [14, 0.0, 0], [15, 1.0, 1], [16, 1.0, 1], [17, 1.0, 1], [18, 1.0, 1], [19, 1.0, 1], [20, 1.0, 1], [21, 1.0, 1], [22, 1.0, 1], [23, 1.0, 1], [24, 1.0, 1], [25, 1.0, 1], [26, 0.0, 1], [27, 1.0, 1], [28, 1.0, 1], [29, 1.0, 1], [30, 0.0, 0], [31, 0.0, 1], [32, 0.0, 1], [33, 0.0, 1], [34, 1.0, 1], [35, 1.0, 1], [36, 0.0, 0], [37, 0.0, 0], [38, 1.0, 1], [39, 1.0, 0], [40, 1.0, 0], [41, 1.0, 1], [42, 1.0, 1], [43, 1.0, 1], [44, 1.0, 1], [45, 1.0, 1], [46, 1.0, 1], [47, 1.0, 1], [48, 1.0, 1], [49, 1.0, 0], [50, 1.0, 1], [51, 0.0, 1], [52, 0.0, 0], [53, 1.0, 1], [54, 1.0, 1], [55, 1.0, 1], [56, 1.0, 1], [57, 1.0, 1], [58, 1.0, 1], [59, 1.0, 0], [60, 0.0, 1], [61, 1.0, 1], [62, 1.0, 1], [63, 1.0, 1], [64, 1.0, 1], [65, 1.0, 1], [66, 1.0, 1], [67, 1.0, 1], [68, 0.0, 0], [69, 1.0, 0